In [ ]:
import json
import uuid
import runpod
import os
import base64
from dotenv import load_dotenv
import asyncio
from pydantic_ai import Agent
import os
from pydantic_ai.models.openai import OpenAIResponsesModelSettings

from consts import IMAGE_GENERATION_OUTPUT_DIR, WORKFLOW_FILE

load_dotenv()
runpod.api_key = os.getenv("RUNPOD_API_KEY")
endpoint = runpod.Endpoint("gl0xwrlwmcvvop")

In [26]:
async def generate_image(prompt: str, short_image_title: str) -> str:
    """Use this tool to generate an image based on a text prompt."""
    IMAGE_GENERATION_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    with open(WORKFLOW_FILE, "r") as f:
        workflow = json.load(f)

    seed = os.urandom(2)
    workflow["31"]["inputs"]["seed"] = int.from_bytes(seed, "big")
    workflow["6"]["inputs"]["text"] = prompt

    payload = {
        "input": {
            "prompt": prompt,
            "seed": 12345,
            "guidance": 7.5,
            "width": 1920,
            "height": 1088,
        }
    }

    run_request = endpoint.run(payload)

    while run_request.status() != "COMPLETED":
        await asyncio.sleep(1)
    result = run_request.output()

    file_uuid = uuid.uuid4().hex
    image_filename = f"{short_image_title}_{file_uuid}.png"

    image = result["image"]

    # for i, image in enumerate(result["images"]):
    with open(IMAGE_GENERATION_OUTPUT_DIR / f"{0}_{image_filename}", "wb") as f:
        f.write(base64.b64decode(image))

    return f"0_{image_filename}"

In [ ]:
PROMPT = """A fantasy forest full of elves, orcs, and dwarves."""

In [36]:
await generate_image(PROMPT, "example_image")

'0_example_image_7ced08fe6f7e46928f35ce8f30e989d4.png'